<a href="https://colab.research.google.com/github/knithinsankar-01/DataHack_Hackthon/blob/main/xyz_and_seasonal_flus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:

# Step 1: Upload the files
print("Please upload 'training_set_features.csv', 'training_set_labels.csv', and 'test_set_features.csv'")
uploaded = files.upload()

Please upload 'training_set_features.csv', 'training_set_labels.csv', and 'test_set_features.csv'


Saving test_set_features.csv to test_set_features (1).csv
Saving training_set_features.csv to training_set_features (1).csv
Saving training_set_labels.csv to training_set_labels (1).csv


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

# Load training data
train_data = pd.read_csv('training_set_features.csv')
labels = pd.read_csv('training_set_labels.csv')
data = train_data.merge(labels, on='respondent_id')

# Separate features (X) and target variables (y)
X = data.drop(['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'], axis=1)
y_xyz = data['xyz_vaccine']
y_seasonal = data['seasonal_vaccine']

# One-hot encode categorical variables in X
X = pd.get_dummies(X, drop_first=True)

# Load test data and perform the same preprocessing
test_data = pd.read_csv('test_set_features.csv')
test_data = pd.get_dummies(test_data, drop_first=True)

# Ensure columns in X and test_data align after one-hot encoding
X, test_data = X.align(test_data, join='left', axis=1, fill_value=0)

# Impute missing values using mean imputation
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
test_data = imputer.transform(test_data)

# Splitting the training data into training and validation sets
X_train, X_val, y_xyz_train, y_xyz_val = train_test_split(X, y_xyz, test_size=0.2, random_state=42)
_, _, y_seasonal_train, y_seasonal_val = train_test_split(X, y_seasonal, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test_data = scaler.transform(test_data)

# Initialize Random Forest classifiers
model_xyz = RandomForestClassifier(n_estimators=100, random_state=42)
model_seasonal = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the models
model_xyz.fit(X_train, y_xyz_train)
model_seasonal.fit(X_train, y_seasonal_train)

# Predict probabilities for validation set
y_xyz_pred = model_xyz.predict_proba(X_val)[:, 1]
y_seasonal_pred = model_seasonal.predict_proba(X_val)[:, 1]

# Calculate ROC AUC scores
roc_auc_xyz = roc_auc_score(y_xyz_val, y_xyz_pred)
roc_auc_seasonal = roc_auc_score(y_seasonal_val, y_seasonal_pred)

# Compute mean ROC AUC score
mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2
print(f'Mean ROC AUC Score: {mean_roc_auc:.4f}')

# Predict probabilities for test set
test_xyz_pred = model_xyz.predict_proba(test_data)[:, 1]
test_seasonal_pred = model_seasonal.predict_proba(test_data)[:, 1]

# Create submission DataFrame
submission = pd.DataFrame({
    'respondent_id': range(len(test_data)),
    'xyz_vaccine': test_xyz_pred,
    'seasonal_vaccine': test_seasonal_pred
})

# Save submission to CSV file
submission.to_csv('submission.csv', index=False)

from google.colab import files
files.download('submission.csv')



Mean ROC AUC Score: 0.8582


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>